In [4]:
# from experiments import generate_dataset
from query_processor import QP_sp_continuity
import matplotlib.pyplot as plt
import networkx as nx
import os
import json
from spreadnet.datasets.data_utils.draw import draw_networkx
from spreadnet.utils import post_processor
from query_processor import QP_sp_continuity as qp_utils
import timeit
from copy import deepcopy
from experiments.hybrid_sp import hybrid_utils
%matplotlib inline

In [5]:
file_2 = os.path.join('predicted_exp1_sp.json')
pred_grp = hybrid_utils.read_json_file(file_2)

file_3 = os.path.join('ground_truth_exp1_sp.json')
gt_grp = hybrid_utils.read_json_file(file_3)

In [6]:
gt_grph_1 = hybrid_utils.read_json_file('gt_exp2.json')
pred_grp_1 = hybrid_utils.read_json_file('pred_exp2.json')

In [ ]:
def extract_path(G):
    """
    _summary_

    _extended_summary_

    Args:
        G (_type_): _description_

    Returns:
        _type_: _description_
    """    
    sp_path = []
    sp_path_edges = []
    sp_pos = {}

    for i in range(0, G.number_of_nodes()):
        sp_pos[i] = G.nodes[i]["pos"]

    for i in range(0, len(G.nodes)):
        if G.nodes[i]["is_in_path"]:
            sp_path.append(i)

    edges_list = list(G.edges(data=True))

    for e in edges_list:
        if e[2]["is_in_path"]:
            sp_path_edges.append([e[0], e[1]])

    return sp_path, sp_path_edges, sp_pos

In [ ]:
def path_continuos_traversal(G,start_node_idx):
        """
        Traverses a simple path based on probabilities in a given graph G; starting form given node start_node_idx. This node can be any node in graph, but the traversal will only look ahead for nodes and edges that are "is_in_path"
        Args:
                G (nx.DiGraph): graph to operate on
                start_node_idx (int): index for the node in G

        Returns:
                visited (list[int]): ordered nodes from start node index
                potential_node (list(int)):nodes which have an in_path incoming edge but node itself is not in_path
                potential_edge (list(int)): nodes which have no in_path incoming edge but are in_path
                disconnected (list(int)): nodes which have no in_path nodes or edges; broken path    
        """         
        visited = [start_node_idx]
        next_visit = visited[-1]
        
        potential_node = []
        potential_edge = []
        disconnected = []
        flag = True
        
        
        while flag and G.nodes[next_visit]['is_in_path']:
                in_cont_path = look_ahead(G,next_visit)
                for u,v,status in in_cont_path :
                        check_v_in_path = G.nodes[v]['is_in_path']
                        if status=='is_connected' or status=='end_is_connected':
                                visited.append(v)
                                next_visit = visited[-1]
                                flag= True
                                break
                        else:
                                if status=='is_potential_node':   
                                        potential_node.append(v)
                                        next_visit = visited[-1]
                                        flag= True
                                        continue  
                                        
                                elif status=='is_potential_edge' and not check_v_in_path:
                                        potential_edge.append(v)
                                        next_visit = visited[-1]
                                        flag= True
                                        continue
                    
                else:
                        if status=='is_disconnected':
                                disconnected.append(u)

                        break
                             
  
        return visited, potential_node, potential_edge, disconnected


In [ ]:
path_continuos_traversal(pred_grp,20)

In [ ]:
path_continuos_traversal(gt_grp,21)

In [ ]:
visited, potential_node, potential_edge, disconnected= path_continuos_traversal(pred_grp, 20)
visited, potential_node, potential_edge, disconnected

In [ ]:
path_continuos_traversal(pred_grp_1,7)

In [ ]:
extract_path(pred_grp_1)

In [ ]:
def eval_path(G,node_idx):
    """
    _summary_

    _extended_summary_

    Args:
        G (_type_): _description_
        node_idx (_type_): _description_
    """
    visited, potential_node, potential_edge, disconnected = path_continuos_traversal(G,node_idx)
    G_nodes, G_edges, G_pos = extract_path(G)
    
    # evaluate visited
    if nx.is_simple_path(G,visited):
        print(f"is simple path")
        #TODO: 1. check if first node is start and last is end
        #2. check if there are potentials to add
        #3. do some checks to them and add in right places
        
        
    else:
        print("not a valid path")
    
    
    
    
    
    

In [ ]:
eval_path(pred_grp_1,node_idx = 7)